In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import re

In [2]:
train = pd.read_csv('zip/train4.csv.gz', compression='gzip', index_col=['id'])
test = pd.read_csv('zip/test4.csv.gz', compression='gzip', index_col=['id'])
train.head()

,product_uid,product_title,search_term,relevance,orig_search_term,orig_product_title,descr,orig_descr,match_product_title,match_descr,...,fuzzy_orig_descr,part_fuzzy_title,part_fuzzy_descr,part_fuzzy_orig_title,part_fuzzy_orig_descr,pred_diff,first_word_title,first_word_descr,match_numbers_title,match_numbers_descr
id,,,,,,,,,,,,,,,,,,,,,
2,100001,simpson strong tie 12 gaug angl,angl bracket,3.00,angle bracket,Simpson Strong-Tie 12-Gauge Angle,"not angl make joint stronger, also provid cons...","Not only do angles make joints stronger, they ...",0.500000,0.500000,...,1,100,100,100,62,0.185792,1,1,0,0
3,100001,simpson strong tie 12 gaug angl,l bracket,2.50,l bracket,Simpson Strong-Tie 12-Gauge Angle,"not angl make joint stronger, also provid cons...","Not only do angles make joints stronger, they ...",0.000000,0.000000,...,1,22,44,33,44,-0.082535,1,1,0,0
9,100002,behr premium textur deckov 1 gal. sc 141 tugbo...,deck,3.00,deck over,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,behr premium textur deckov innov solid color c...,BEHR Premium Textured DECKOVER is an innovativ...,0.000000,0.000000,...,62,100,100,89,100,0.248623,1,1,0,0
16,100005,delta vero 1 handl shower faucet trim kit chro...,rain shower head,2.33,rain shower head,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,updat bathroom delta vero single-handl shower ...,Update your bathroom with the Delta Vero Singl...,0.333333,0.333333,...,55,100,100,100,100,0.027899,0,0,0,0
17,100005,delta vero 1 handl shower faucet trim kit chro...,shower faucet,2.67,shower only faucet,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,updat bathroom delta vero single-handl shower ...,Update your bathroom with the Delta Vero Singl...,1.000000,1.000000,...,100,100,100,100,100,-0.111435,1,1,0,0


In [3]:
pd.set_option('display.max_colwidth', -1)
train.sort_values(['pred_diff'], ascending=False)[:50]

,product_uid,product_title,search_term,relevance,orig_search_term,orig_product_title,descr,orig_descr,match_product_title,match_descr,...,fuzzy_orig_descr,part_fuzzy_title,part_fuzzy_descr,part_fuzzy_orig_title,part_fuzzy_orig_descr,pred_diff,first_word_title,first_word_descr,match_numbers_title,match_numbers_descr
id,,,,,,,,,,,,,,,,,,,,,
193001,183692,lichtenberg white no. 918 millenni ryan heather textur sheer curtain panel 40in. w xbi 95in. l,w g 918,3.00,w g 918,"LICHTENBERG White No. 918 Millennial Ryan Heathered Texture Sheer Curtain Panel, 40 in. W x 95 in. L","no. 918 millenni ryan heather textur semi-sh curtain casual solid add fresh finish touch decor setting. enhanc privaci allow light gentl filter through. clean, simpl one-pocket pole top design use standard decor curtain rod. mix match solid print look own.sheer panel, gentl filter lightno header pole top panelmachin washablewid array color choos from100% polyestercontain 1-curtain panel","No. 918 Millennial Ryan heathered texture semi-sheer curtain is a casual solid that adds freshness and a finishing touch to any decor setting. Enhances privacy while allowing light to gently filter through. Clean, simple one-pocket pole top design can be used with a standard or decorative curtain rod. Mix and match with other solids and prints for a look that is all your own.Sheer panel, gently filters lightNo header pole top panelMachine washableWide array of colors to choose from100% polyesterContains 1-curtain panel",0.666667,0.333333,...,60,100,100,100,100,1.168968,1,1,1,1
203914,192088,behr premium plus 600e 2 harbor mist zero voc interior paint,locit 2 plus 1,3.00,locite 2 plus 1,BEHR Premium Plus #600E-2 Harbor Mist Zero VOC Interior Paint,"behr premium plus zero voc, self-prim interior flat 100% acryl provid mildew resist finish. this easy-clean flat provid excel touch-up, ultim durabl except hide. the finish display non-reflective, matt appearance.california residents: see&nbsp;proposit 65 informationid famili rooms, live rooms, dine rooms, bedroom ceilings100% acryl finish easi cleaningtough durabl matt finishexcept hide, 1 cover 400 sq. ft. depend color select surfac porosityeasi soap water clean-upexcept latex formula provid mildew resist finishself prime finish seal surfaceactu paint color may vari on-screen printer representationsonlin price includ paint care fee follow states: ca, co, ct, me, mn, or, ri, vt","BEHR Premium Plus Zero VOC, Self-Priming Interior Flat is 100% acrylic and provides a mildew resistant finish. This easy-clean flat provides excellent touch-up, ultimate durability and exceptional hide. The finish displays a non-reflective, matte appearance.California residents: see&nbsp;Proposition 65 informationIdeal for family rooms, living rooms, dining rooms, bedrooms and ceilings100% acrylic finish for easy cleaningTough and durable matte finishExceptional hide, 1 can covers up to 400 sq. ft. depending on color selected and surface porosityEasy soap and water clean-upExceptional latex formula provides a mildew resistant finishSelf priming finish that seals the surfaceActual paint colors may vary from on-screen and printer representationsOnline Price includes Paint Care fee in the following states: CA, CO, CT, ME, MN, OR, RI, VT",0.500000,0.500000,...,57,100,100,100,100,1.134340,0,0,0,0
194147,184559,lenmar nickel metal hydrid 1200mah 3.6 volt cordless phone replac batteri,polish batteri metal,3.00,polisher battery metal,Lenmar Nickel-Metal Hydride 1200mAh/3.6-Volt Cordless Phone Replacement Battery,"power panasonic, american telecom, tele-phone, key phone, audiovox, uniden, universal, zeta cordless phone lenmar nickel-met hydrid cordless phone replac battery. this 3.6-volt/1200mah batteri provid reliable, long-last power. compat panason kx-a150, american telecom 2110, tele-phon tel-1000, key phone 920-wd, audiovox at-12a, uniden ex-3300, univers 2000, zeta 510.fits: panason kx-a150, american telecom 2110, tele-phon tel-1000, key phone 920-wd, audiovox a

In [17]:
@np.vectorize
def split_descr_words(s):
    s = s.replace('&nbsp;', ' ')
    s = re.sub(r"([a-z])([\$A-Z0-9])", r"\1 \2", s)
    s = re.sub(r"([a-z])[\.\:]([A-Z])", r"\1 \2", s)
    return s


In [25]:
stemmer = SnowballStemmer('english')
stop_words = set(stopwords.words('english'))
stop_words.add('xby')


def replaces(s):
    if isinstance(s, str):
        s = s.lower()
        s = re.sub(r"(\w)[\.:]([A-Z])", r"\1 \2", s)
    
        s = s.replace(" x "," xby ")
        s = s.replace("*"," xby ")
        s = s.replace(" by "," xby")
        s = s.replace("x0"," xby 0")
        s = s.replace("x1"," xby 1")
        s = s.replace("x2"," xby 2")
        s = s.replace("x3"," xby 3")
        s = s.replace("x4"," xby 4")
        s = s.replace("x5"," xby 5")
        s = s.replace("x6"," xby 6")
        s = s.replace("x7"," xby 7")
        s = s.replace("x8"," xby 8")
        s = s.replace("x9"," xby 9")
        s = s.replace("0x","0 xby ")
        s = s.replace("1x","1 xby ")
        s = s.replace("2x","2 xby ")
        s = s.replace("3x","3 xby ")
        s = s.replace("4x","4 xby ")
        s = s.replace("5x","5 xby ")
        s = s.replace("6x","6 xby ")
        s = s.replace("7x","7 xby ")
        s = s.replace("8x","8 xby ")
        s = s.replace("9x","9 xby ")
        
        s = re.sub(r"([0-9])( *)\.( *)([0-9])", r"\1.\4", s)
        s = re.sub(r"([0-9]+)( *)(inches|inch|in|')\.?[^a-zA-z]", r"\1in. ", s)
        s = re.sub(r"([0-9]+)( *)(foot|feet|ft|'')\.?", r"\1ft. ", s)
        s = re.sub(r"([0-9]+)( *)(pounds|pound|lbs|lb)\.?", r"\1lb. ", s)
        s = re.sub(r"([0-9]+)( *)(square|sq) ?\.?(feet|foot|ft)\.?", r"\1sq.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(cubic|cu) ?\.?(feet|foot|ft)\.?", r"\1cu.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(gallons|gallon|gal)\.?", r"\1gal. ", s)
        s = re.sub(r"([0-9]+)( *)(ounces|ounce|oz)\.?", r"\1oz. ", s)
        s = re.sub(r"([0-9]+)( *)(centimeters|cm)\.?", r"\1cm. ", s)
        s = re.sub(r"([0-9]+)( *)(milimeters|mm)\.?", r"\1mm. ", s)
        s = s.replace("°"," degrees ")
        s = re.sub(r"([0-9]+)( *)(degrees|degree)\.?", r"\1deg. ", s)
        s = s.replace(" v "," volts ")
        s = re.sub(r"([0-9]+)( *)(volts|volt)\.?", r"\1volt. ", s)
        s = re.sub(r"([0-9]+)( *)(watts|watt)\.?", r"\1watt. ", s)
        s = re.sub(r"([0-9]+)( *)(amperes|ampere|amps|amp)\.?", r"\1amp. ", s)
        
        s = s.replace("toliet", "toilet")
        s = s.replace("airconditioner", "air conditioner")
        s = s.replace("vinal", "vinyl")
        s = s.replace("vynal", "vinyl")
        s = s.replace("skill", "skil")
        s = s.replace("snowbl", "snow bl")
        s = s.replace("plexigla", "plexi gla")
        s = s.replace("rustoleum", "rust-oleum")
        s = s.replace("whirpool", "whirlpool")
        s = s.replace("whirlpoolga", "whirlpool ga")
        s = s.replace("whirlpoolstainless", "whirlpool stainless")
        s = s.replace("unsulation", "insulation")

        s = s.replace("  "," ")
        
        s = s.replace("/", " ")
        s = s.replace("-", " ")
        return " ".join([re.sub('[^A-Za-z0-9./]', ' ', word)
                         for word in s.split()])
    else:
        return "null"

@np.vectorize
def stem_sentence(s): 
    s = replaces(s)   
    return str(" ".join([stemmer.stem(word)
                         for word in s.split()
                         if word not in stop_words]))


In [26]:
encoding = "ISO-8859-1"

products = pd.read_csv('zip/product_descriptions.csv.gz', encoding=encoding, compression='gzip',
                       index_col=['product_uid'])
products['orig_descr'] = split_descr_words(products['product_description'].astype(str))
products['descr'] = stem_sentence(products['orig_descr'])


In [27]:
products = products.drop(columns=['product_description'])
products.to_csv('zip/descr2.csv.gz', compression='gzip')

In [28]:
train['descr'] = train.product_uid.map(products['descr'])
train['orig_descr'] = train.product_uid.map(products['orig_descr'])

train.sort_values(['pred_diff'], ascending=False)[:50]

,product_uid,product_title,search_term,relevance,orig_search_term,orig_product_title,descr,orig_descr,match_product_title,match_descr,...,fuzzy_orig_descr,part_fuzzy_title,part_fuzzy_descr,part_fuzzy_orig_title,part_fuzzy_orig_descr,pred_diff,first_word_title,first_word_descr,match_numbers_title,match_numbers_descr
id,,,,,,,,,,,,,,,,,,,,,
193001,183692,lichtenberg white no. 918 millenni ryan heather textur sheer curtain panel 40in. w xbi 95in. l,w g 918,3.00,w g 918,"LICHTENBERG White No. 918 Millennial Ryan Heathered Texture Sheer Curtain Panel, 40 in. W x 95 in. L",no. 918 millenni ryan heather textur semi sheer curtain casual solid add fresh finish touch decor setting. enhanc privaci allow light gentl filter through. clean simpl one pocket pole top design use standard decor curtain rod. mix match solid print look sheer panel gentl filter light header pole top panel machin washabl wide array color choos 100 polyest contain 1 curtain panel,"No. 918 Millennial Ryan heathered texture semi-sheer curtain is a casual solid that adds freshness and a finishing touch to any decor setting. Enhances privacy while allowing light to gently filter through. Clean, simple one-pocket pole top design can be used with a standard or decorative curtain rod. Mix and match with other solids and prints for a look that is all your own Sheer panel, gently filters light No header pole top panel Machine washable Wide array of colors to choose from 100% polyester Contains 1-curtain panel",0.666667,0.333333,...,60,100,100,100,100,1.168968,1,1,1,1
203914,192088,behr premium plus 600e 2 harbor mist zero voc interior paint,locit 2 plus 1,3.00,locite 2 plus 1,BEHR Premium Plus #600E-2 Harbor Mist Zero VOC Interior Paint,behr premium plus zero voc self prime interior flat 100 acryl provid mildew resist finish. easi clean flat provid excel touch ultim durabl except hide. finish display non reflect matt appear california resid see proposit 65 inform ideal famili room live room dine room bedroom ceil 100 acryl finish easi clean tough durabl matt finish except hide 1 cover 400 sq. ft. depend color select surfac poros easi soap water clean except latex formula provid mildew resist finish self prime finish seal surfac actual paint color may vari screen printer represent onlin price includ paint care fee follow state ca co ct mn ri vt,"BEHR Premium Plus Zero VOC, Self-Priming Interior Flat is 100% acrylic and provides a mildew resistant finish. This easy-clean flat provides excellent touch-up, ultimate durability and exceptional hide. The finish displays a non-reflective, matte appearance California residents: see Proposition 65 information Ideal for family rooms, living rooms, dining rooms, bedrooms and ceilings 100% acrylic finish for easy cleaning Tough and durable matte finish Exceptional hide, 1 can covers up to 400 sq. ft. depending on color selected and surface porosity Easy soap and water clean-up Exceptional latex formula provides a mildew resistant finish Self priming finish that seals the surface Actual paint colors may vary from on-screen and printer representations Online Price includes Paint Care fee in the following states: CA, CO, CT, ME, MN, OR, RI, VT",0.500000,0.500000,...,57,100,100,100,100,1.134340,0,0,0,0
194147,184559,lenmar nickel metal hydrid 1200mah 3.6 volt cordless phone replac batteri,polish batteri metal,3.00,polisher battery metal,Lenmar Nickel-Metal Hydride 1200mAh/3.6-Volt Cordless Phone Replacement Battery,power panason american telecom tele phone key phone audiovox uniden univers zeta cordless phone lenmar nickel metal hydrid cordless phone replac battery. 3.6 volt 1200m ah batteri provid reliabl long last power. compat panason kx a150 american telecom 2110 tele phone tel 1000 key phone 920 wd audiovox 12a uniden ex 3300 univers 2000 zeta 510.fit panason kx a150 american telecom 2110 tele phone tel 1000 key phone 920 wd audiovox 12a uniden ex 3300 univers 2000 zeta 510replac dantona 3aa empir cpb 400 general electr bt 17 panason p 

In [29]:
train.sort_values(['pred_diff'], ascending=True)[:50]

,product_uid,product_title,search_term,relevance,orig_search_term,orig_product_title,descr,orig_descr,match_product_title,match_descr,...,fuzzy_orig_descr,part_fuzzy_title,part_fuzzy_descr,part_fuzzy_orig_title,part_fuzzy_orig_descr,pred_diff,first_word_title,first_word_descr,match_numbers_title,match_numbers_descr
id,,,,,,,,,,,,,,,,,,,,,
69679,119478,romano 4ft. boxwood spiral topiari tree,topiari tree,1.00,topiary tree,Romano 4 ft. Boxwood Spiral Topiary Tree,enhanc home romano boxwood spiral topiari tree. wonder full bodi boxwood spiral topiari tree keep color rest lose color leaves. alway look forward rich green tone romano boxwood spiral topiari tree home. look high qualiti artifici tree make bold statement home light mobil design 700 leav indoor outdoor iron frame uv resist rubber leav 7in. pot enhanc room hous,"Enhance your home with a Romano Boxwood Spiral Topiary Tree. This wonderful full bodied Boxwood Spiral Topiary Tree keeps its colors when all the rest lose their color and leaves. You can always look forward to the rich green tones of the Romano Boxwood Spiral Topiary Tree in your home. If you are looking for a High quality artificial tree to make a bold statement in your home, this is it Light and mobile design Over 700 leaves Indoor/outdoor Iron frame UV resistant rubber leaves 7 in. pot Enhances any room in your house",1.000000,1.000000,...,100,100,100,100,100,-1.594096,1,1,0,0
136808,148897,earthquak 212cc tiller rear tine crt side shield,side shield,1.00,side shields,Earthquake 212cc Tiller Rear Tine CRT with Side Shields,earthquak 6015v rear tine rototil deliv ultim combin power size. rear mount tine till stubborn dirt easili high perform 212cc viper engine. engin compact rear tine rototil power larger counterpart easi singl hand control sod earthquak 6015v rear tine compact full size power compact frame durabl cast iron bronz gear drive transmiss,"The Earthquake 6015V rear tine rototiller delivers the ultimate combination of power and size. Rear-mounted tines till stubborn dirt easily with a high-performance 212cc Viper engine. We have engineered a compact rear tine rototiller with the same power as its larger counterparts Easy - single-handed control through sod Earthquake 6015V rear tine Compact - full-sized power in a compact frame Durable - cast iron, bronze gear drive transmission",1.000000,0.000000,...,0,100,64,100,67,-1.426482,1,0,0,0
120624,140844,rachael ray 10 qt. cover stockpot,rachael ray,1.00,rachael ray,Rachael Ray 10 qt. Covered Stockpot,whether boil pasta make batch chili cook grain larg 10 quart stockpot right size. stovetop two short side handl help take less space cabinet. hard anod construct prefer even heat distribut throughout pan sides. nonstick cook surfac mean need worri food stick scrub dirti pan cleanup breeze. fact rachael stockpot dishwash safe ad conveni hard anod cookwar except durabl plus heat quick even reduc hot spot burn food clear coat exterior make cookwar dishwash safe conveni long last nonstick interior let food slide eas make cleanup quick easi fun rubber handl provid comfort grip oven safe 350deg. fahrenheit temper glass lid watch food cook without remov lid keep heat moistur lock lifetim limit warranti,"Whether you're boiling pasta, making a batch of chili or cooking grains, this large, 10 quart stockpot is just the right size. When it's not on the stovetop, its two short side handles help it take up less space in your cabinet. The hard-anodized construction is preferred for its even heat distribution throughout the pan and up the sides. The nonstick cooking surface means you don't need to worry about food sticking or about scrubbing dirty pans - cleanup is a breeze. In fact, Rachael's stockpot is dishwasher safe for added convenience Hard-anodized cookware is exceptionally durable, plus it heats quickly and evenly, reducing ""hot spots"" that can burn foods A clear coat exterior makes this cookware dishwasher safe for convenience Long-lasting nonstick interior let

In [30]:
test['descr'] = test.product_uid.map(products['descr'])
test['orig_descr'] = test.product_uid.map(products['orig_descr'])

In [31]:
print(train.columns)

Index(['product_uid', 'product_title', 'search_term', 'relevance',
       'orig_search_term', 'orig_product_title', 'descr', 'orig_descr',
       'match_product_title', 'match_descr', 'len_search_term_words',
       'len_search_term_letters', 'complete_match_title',
       'complete_match_descr', 'brand', 'match_brand', 'words_std_title',
       'words_std_descr', 'last_word_title', 'last_word_descr',
       'prod_title_ind', 'descr_ind', 'match_pos_title', 'match_pos_descr',
       'match_orig_title', 'match_orig_descr', 'len_title', 'len_descr',
       'fuzzy_title', 'fuzzy_descr', 'fuzzy_orig_title', 'fuzzy_orig_descr',
       'part_fuzzy_title', 'part_fuzzy_descr', 'part_fuzzy_orig_title',
       'part_fuzzy_orig_descr', 'pred_diff', 'first_word_title',
       'first_word_descr', 'match_numbers_title', 'match_numbers_descr'],
      dtype='object')


In [32]:
from fuzzywuzzy import fuzz


@np.vectorize
def word_match_count(search_term, text):
    count = 0
    search_term = search_term.split()
    text = text.split()
    if len(search_term) == 0:
        return 0
    for word in set(search_term):
        count += 1 if text.count(word) > 0 else 0
    return count / len(search_term)


@np.vectorize
def word_match_std(search_term, text):
    indexes = []
    search_term = search_term.split()
    text = text.split()    
    n = len(text)
    if len(search_term) == 0:
        return 0
    for word in set(search_term):
        if word in text:
            indexes.append(text.index(word))
    if len(indexes) > 0:
        return np.var(indexes)
    else:
        return 0

    
@np.vectorize
def match_last_word(search_term, text):
    last_word = search_term.split()[-1]
    if text.find(last_word) > -1:
        return 1
    else:
        return 0


@np.vectorize    
def word_match_index(search_term, text):
    count = 0
    search_term = search_term.split()
    text = text.split()
    if len(search_term) == 0:
        return 0
    min_ind = len(text)
    for word in set(search_term):
        if word in text:
            min_ind = min(min_ind, text.index(word))
    return min_ind / len(text) 


@np.vectorize 
def match_word_n_pos(text, search_term):
    text = nltk.pos_tag(replaces(text).split())
    search_pos = nltk.pos_tag(replaces(search_term).split())
    text = ["%s/%s" % (word[0].lower(), word[1]) for word in text]
    search_pos = ["%s/%s" % (word[0].lower(), word[1]) for word in search_pos]
    
    if len(search_pos) == 0:
        return 0
    matches = 0
    for word in set(text):
        if word in search_pos:
            matches += 1
    return matches / len(search_pos)


@np.vectorize
def match_first_word(search_term, text):
    last_word = search_term.split()[0]
    if text.find(last_word) > -1:
        return 1
    else:
        return 0

    
@np.vectorize
def match_numbers(search_term, text):
    search_term_numbers = set(re.findall(r'\d+', search_term))
    len_ = len(search_term_numbers)
    if len_ == 0:
        return 0
    text_numbers = set(re.findall(r'\d+', text))
    match = sum([1 for i in search_term_numbers if i in text_numbers else 0])
    return match / len_


@np.vectorize
def fuzzy_match(search_term, field):
    if search_term == 'null':
        return 0
    return fuzz.token_set_ratio(search_term, field)


@np.vectorize
def fuzzy_match_partial(search_term, field):
    if search_term == 'null':
        return 0
    return fuzz.partial_token_set_ratio(search_term, field)

In [46]:
%%time
train['search_term'] = stem_sentence(train['orig_search_term'].astype(str) )
train.loc[train['search_term'] == '', 'search_term'] = 'null'

train['product_title'] = stem_sentence(train['orig_product_title'].astype(str) )

for col in ['product_title', 'descr']:
    train['match_' + col] = word_match_count(train['search_term'], train[col])
    
train['len_search_term_words'] = train['search_term'].apply(lambda x: len(x.split()))
train['len_search_term_letters'] = train['search_term'].str.len()

train['complete_match_title'] = \
    train.apply(lambda row:1 if row['product_title'].find(row['search_term']) >= 0 else 0, axis=1)   
train['complete_match_descr'] = \
    train.apply(lambda row:1 if row['descr'].find(row['search_term']) >= 0 else 0, axis=1)

train['brand'] = train['brand'].astype(str)
train['match_brand'] = word_match_count(train['brand'], train['search_term'])

train['words_std_title'] = word_match_std(train['search_term'], train['product_title'])
train['words_std_descr'] = word_match_std(train['search_term'], train['descr'])

train['last_word_title'] = match_last_word(train['search_term'], train['product_title'])  
train['last_word_descr'] = match_last_word(train['search_term'], train['descr']) 

train['prod_title_ind'] = word_match_index(train['search_term'], train['product_title'])
train['descr_ind'] = word_match_index(train['search_term'], train['descr'])

train['match_pos_title'] = match_word_n_pos(train['orig_product_title'], train['orig_search_term'])
train['match_pos_descr'] = match_word_n_pos(train['orig_descr'], train['orig_search_term'])

train['match_orig_title'] = word_match_count(train['search_term'], train['orig_product_title'])
train['match_orig_descr'] = word_match_count(train['search_term'], train['orig_descr'])

train['len_title'] = train['product_title'].apply(lambda x: len(x.split()))
train['len_descr'] = train['descr'].apply(lambda x: len(x.split()))

train['first_word_title'] = match_first_word(train['search_term'], train['product_title'])  
train['first_word_descr'] = match_first_word(train['search_term'], train['descr']) 

train['match_numbers_title'] = match_numbers(train['search_term'], train['product_title'])  
train['match_numbers_descr'] = match_numbers(train['search_term'], train['descr']) 

train['fuzzy_title'] = fuzzy_match(train['search_term'], train['product_title'])
train['fuzzy_descr'] = fuzzy_match(train['search_term'], train['descr'])

train['fuzzy_orig_title'] = fuzzy_match(train['orig_search_term'], train['orig_product_title'])
train['fuzzy_orig_descr'] = fuzzy_match(train['orig_search_term'], train['orig_descr'])

CPU times: user 8min 3s, sys: 1.4 s, total: 8min 4s
Wall time: 8min 4s


In [47]:
train.to_csv('zip/train5.csv.gz', compression='gzip')

In [36]:
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion

In [37]:
class cust_regression_vals(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self
    def transform(self, train):
        return train[tree_cols].values


class cust_txt_col(BaseEstimator, TransformerMixin):
    def __init__(self, col):
        self.col = col
    def fit(self, x, y=None):
        return self
    def transform(self, dataset):
        return dataset[self.col].apply(str)

In [39]:
def fitter():
    clf = Pipeline([
    ('union', FeatureUnion(
                transformer_list = [
                    ('cst',  cust_regression_vals()),  
                    ('txt1', Pipeline([('s1', cust_txt_col(col='search_term')),
                                       ('tfidf1', TfidfVectorizer(stop_words='english')),
                                       ('tsvd1', TruncatedSVD(n_components=80))])),
                    ('txt2', Pipeline([('s2', cust_txt_col(col='product_title')),
                                       ('tfidf2', TfidfVectorizer(stop_words='english')),
                                       ('tsvd2', TruncatedSVD(n_components=80))])),
                    ('txt3', Pipeline([('s3', cust_txt_col(col='descr')),
                                       ('tfidf3', TfidfVectorizer(stop_words='english')),
                                       ('tsvd3', TruncatedSVD(n_components=60))])),
                    ('txt4', Pipeline([('s4', cust_txt_col(col='brand')),
                                       ('tfidf4', TfidfVectorizer(stop_words='english')),
                                       ('tsvd4', TruncatedSVD(n_components=30))]))
                    ]
            ))])
    clf.fit(train[use_cols])
    return clf

In [48]:
tree_cols = ['match_product_title', 'match_descr', 'len_search_term_words', 'len_search_term_letters',
             'match_pos_title', 'match_brand', 'match_pos_descr', 'prod_title_ind', 'descr_ind',
             'words_std_title', 'words_std_descr', 'len_title', 'len_descr', 'last_word_title',
             'last_word_descr', 'match_orig_title', 'match_orig_descr',
             'fuzzy_title', 'fuzzy_descr', 'fuzzy_orig_title', 'fuzzy_orig_descr',
             'part_fuzzy_title', 'part_fuzzy_descr', 'part_fuzzy_orig_title', 'part_fuzzy_orig_descr',
             'first_word_title', 'first_word_descr',
             'match_numbers_title']

use_cols = tree_cols + ['search_term', 'product_title', 'descr', 'brand']

y = train['relevance'].values

clf = fitter()
X_train = clf.transform(train[use_cols])

gbm = lgb.LGBMRegressor(learning_rate=0.03, colsample_bytree=0.5, num_leaves=80, n_estimators=1000)
gbm.fit(X_train, y)

tc_len = len(tree_cols)
print(dict(zip(tree_cols, gbm.feature_importances_[:tc_len]) ))
print(gbm.score(X_train, y))


{'match_product_title': 237, 'match_descr': 150, 'len_search_term_words': 253, 'len_search_term_letters': 602, 'match_pos_title': 193, 'match_brand': 57, 'match_pos_descr': 154, 'prod_title_ind': 633, 'descr_ind': 446, 'words_std_title': 229, 'words_std_descr': 287, 'len_title': 377, 'len_descr': 355, 'last_word_title': 94, 'last_word_descr': 62, 'match_orig_title': 118, 'match_orig_descr': 153, 'fuzzy_title': 542, 'fuzzy_descr': 399, 'fuzzy_orig_title': 520, 'fuzzy_orig_descr': 421, 'part_fuzzy_title': 174, 'part_fuzzy_descr': 141, 'part_fuzzy_orig_title': 210, 'part_fuzzy_orig_descr': 71, 'first_word_title': 109, 'first_word_descr': 51, 'match_numbers_title': 155}
0.6510599346923697


In [49]:
%%time
test['search_term'] = stem_sentence(test['orig_search_term'].astype(str) )
test.loc[test['search_term'] == '', 'search_term'] = 'null'

train['product_title'] = stem_sentence(train['orig_product_title'].astype(str) )

for col in ['product_title', 'descr']:
    test['match_' + col] = word_match_count(test['search_term'], test[col])
    
test['len_search_term_words'] = test['search_term'].apply(lambda x: len(x.split()))
test['len_search_term_letters'] = test['search_term'].str.len()

test['complete_match_title'] = \
    test.apply(lambda row:1 if row['product_title'].find(row['search_term']) >= 0 else 0, axis=1)   
test['complete_match_descr'] = \
    test.apply(lambda row:1 if row['descr'].find(row['search_term']) >= 0 else 0, axis=1)

test['brand'] = test['brand'].astype(str)
test['match_brand'] = word_match_count(test['brand'], test['search_term'])

test['words_std_title'] = word_match_std(test['search_term'], test['product_title'])
test['words_std_descr'] = word_match_std(test['search_term'], test['descr'])

test['last_word_title'] = match_last_word(test['search_term'], test['product_title'])  
test['last_word_descr'] = match_last_word(test['search_term'], test['descr']) 

test['prod_title_ind'] = word_match_index(test['search_term'], test['product_title'])
test['descr_ind'] = word_match_index(test['search_term'], test['descr'])

test['match_pos_title'] = match_word_n_pos(test['orig_product_title'], test['orig_search_term'])
test['match_pos_descr'] = match_word_n_pos(test['orig_descr'], test['orig_search_term'])

test['match_orig_title'] = word_match_count(test['search_term'], test['orig_product_title'])
test['match_orig_descr'] = word_match_count(test['search_term'], test['orig_descr'])

test['len_title'] = test['product_title'].apply(lambda x: len(x.split()))
test['len_descr'] = test['descr'].apply(lambda x: len(x.split()))

test['first_word_title'] = match_first_word(test['search_term'], test['product_title'])  
test['first_word_descr'] = match_first_word(test['search_term'], test['descr']) 

test['match_numbers_title'] = match_numbers(test['search_term'], test['product_title'])  
test['match_numbers_descr'] = match_numbers(test['search_term'], test['descr']) 

test['fuzzy_title'] = fuzzy_match(test['search_term'], test['product_title'])
test['fuzzy_descr'] = fuzzy_match(test['search_term'], test['descr'])

test['fuzzy_orig_title'] = fuzzy_match(test['orig_search_term'], test['orig_product_title'])
test['fuzzy_orig_descr'] = fuzzy_match(test['orig_search_term'], test['orig_descr'])

CPU times: user 17min 51s, sys: 2.85 s, total: 17min 54s
Wall time: 17min 54s


In [50]:
test.to_csv('zip/test5.csv.gz', compression='gzip')

In [51]:
X_test = clf.transform(test[use_cols])
y_pred = gbm.predict(X_test)
y_pred[y_pred>3.] = 3.
y_pred[y_pred<1.] = 1.
results = pd.DataFrame({'id':test.index.values, 'relevance':y_pred})
results.to_csv('lgbm5.csv', header=True, index=False)

0.46781

In [52]:
train['pred_diff'] = train['relevance'] - gbm.predict(X_train)
train.sort_values(['pred_diff'], ascending=False)[:50]

,product_uid,product_title,search_term,relevance,orig_search_term,orig_product_title,descr,orig_descr,match_product_title,match_descr,...,fuzzy_orig_descr,part_fuzzy_title,part_fuzzy_descr,part_fuzzy_orig_title,part_fuzzy_orig_descr,pred_diff,first_word_title,first_word_descr,match_numbers_title,match_numbers_descr
id,,,,,,,,,,,,,,,,,,,,,
203914,192088,behr premium plus 600e 2 harbor mist zero voc interior paint,locit 2 plus 1,3.00,locite 2 plus 1,BEHR Premium Plus #600E-2 Harbor Mist Zero VOC Interior Paint,behr premium plus zero voc self prime interior flat 100 acryl provid mildew resist finish. easi clean flat provid excel touch ultim durabl except hide. finish display non reflect matt appear california resid see proposit 65 inform ideal famili room live room dine room bedroom ceil 100 acryl finish easi clean tough durabl matt finish except hide 1 cover 400 sq. ft. depend color select surfac poros easi soap water clean except latex formula provid mildew resist finish self prime finish seal surfac actual paint color may vari screen printer represent onlin price includ paint care fee follow state ca co ct mn ri vt,"BEHR Premium Plus Zero VOC, Self-Priming Interior Flat is 100% acrylic and provides a mildew resistant finish. This easy-clean flat provides excellent touch-up, ultimate durability and exceptional hide. The finish displays a non-reflective, matte appearance California residents: see Proposition 65 information Ideal for family rooms, living rooms, dining rooms, bedrooms and ceilings 100% acrylic finish for easy cleaning Tough and durable matte finish Exceptional hide, 1 can covers up to 400 sq. ft. depending on color selected and surface porosity Easy soap and water clean-up Exceptional latex formula provides a mildew resistant finish Self priming finish that seals the surface Actual paint colors may vary from on-screen and printer representations Online Price includes Paint Care fee in the following states: CA, CO, CT, ME, MN, OR, RI, VT",0.500000,0.500000,...,57,100,100,100,100,1.102969,0,0,0,0
185512,178376,behr premium plus ultra 1 gal. p460 5 fiji semi gloss enamel exterior paint,fija,3.00,fija,BEHR Premium Plus Ultra 1-gal. #P460-5 Fiji Semi-Gloss Enamel Exterior Paint,perfect pop color give home serious curb appeal choos behr premium plus ultra semi gloss enamel exterior paint. radiant sheen creat sleek appear home exterior accent ideal trim shutter door includ garag door well window outdoor furniture. formul withstand wear also resist moistur fade stain california resid see proposit 65 inform suitabl applic wood vinyl stucco aluminum metal steel door window trim shutter garag door outdoor furnitur rail wrought iron 100 acryl base provid except adhes durabl extra protect shell guard damag caus xbyuv ray sever weather condit long last fresh paint look minim mainten enhanc mildew resist finish except hide 1 cover 400 sq. ft. depend color select surfac poros semi gloss enamel sheen offer radiant sleek appear formul easi clean soap water actual paint color may vari screen printer represent onlin price includ paint care fee follow state ca co ct mn ri vt,"For a perfect pop of color that will give your home some serious curb appeal, choose BEHR Premium Plus Ultra Semi-Gloss Enamel Exterior Paint. This radiant sheen will create a sleek appearance on your home's exterior accents and is ideal for trim, shutters and doors, including garage doors, as well as windows and outdoor furniture. It is formulated to withstand wear and will also resist moisture, fading and stains California residents: see Proposition 65 information Suitable for application on wood, vinyl, stucco, aluminum, metals, steel, doors, windows, trim, shutters, garage doors, outdoor furniture, railing and wrought iron 100% acrylic base provides exceptional adhesion and durability Extra-protective shell guards against damage caused by UV rays and severe weather conditions for a long-lasting freshly painted look with minimal maintenance Enhanced mildew res

In [53]:
train.sort_values(['pred_diff'], ascending=True)[:50]

,product_uid,product_title,search_term,relevance,orig_search_term,orig_product_title,descr,orig_descr,match_product_title,match_descr,...,fuzzy_orig_descr,part_fuzzy_title,part_fuzzy_descr,part_fuzzy_orig_title,part_fuzzy_orig_descr,pred_diff,first_word_title,first_word_descr,match_numbers_title,match_numbers_descr
id,,,,,,,,,,,,,,,,,,,,,
136808,148897,earthquak 212cc tiller rear tine crt side shield,side shield,1.00,side shields,Earthquake 212cc Tiller Rear Tine CRT with Side Shields,earthquak 6015v rear tine rototil deliv ultim combin power size. rear mount tine till stubborn dirt easili high perform 212cc viper engine. engin compact rear tine rototil power larger counterpart easi singl hand control sod earthquak 6015v rear tine compact full size power compact frame durabl cast iron bronz gear drive transmiss,"The Earthquake 6015V rear tine rototiller delivers the ultimate combination of power and size. Rear-mounted tines till stubborn dirt easily with a high-performance 212cc Viper engine. We have engineered a compact rear tine rototiller with the same power as its larger counterparts Easy - single-handed control through sod Earthquake 6015V rear tine Compact - full-sized power in a compact frame Durable - cast iron, bronze gear drive transmission",1.000000,0.000000,...,7,100,64,100,67,-1.510562,1,0,0,0
120624,140844,rachael ray 10 qt. cover stockpot,rachael ray,1.00,rachael ray,Rachael Ray 10 qt. Covered Stockpot,whether boil pasta make batch chili cook grain larg 10 quart stockpot right size. stovetop two short side handl help take less space cabinet. hard anod construct prefer even heat distribut throughout pan sides. nonstick cook surfac mean need worri food stick scrub dirti pan cleanup breeze. fact rachael stockpot dishwash safe ad conveni hard anod cookwar except durabl plus heat quick even reduc hot spot burn food clear coat exterior make cookwar dishwash safe conveni long last nonstick interior let food slide eas make cleanup quick easi fun rubber handl provid comfort grip oven safe 350deg. fahrenheit temper glass lid watch food cook without remov lid keep heat moistur lock lifetim limit warranti,"Whether you're boiling pasta, making a batch of chili or cooking grains, this large, 10 quart stockpot is just the right size. When it's not on the stovetop, its two short side handles help it take up less space in your cabinet. The hard-anodized construction is preferred for its even heat distribution throughout the pan and up the sides. The nonstick cooking surface means you don't need to worry about food sticking or about scrubbing dirty pans - cleanup is a breeze. In fact, Rachael's stockpot is dishwasher safe for added convenience Hard-anodized cookware is exceptionally durable, plus it heats quickly and evenly, reducing ""hot spots"" that can burn foods A clear coat exterior makes this cookware dishwasher safe for convenience Long-lasting nonstick interior lets food slide off with ease and makes cleanup quick and easy Fun rubberized handles provide a comfortable grip and are oven safe to 350 degrees Fahrenheit Tempered glass lid to watch your foods cook without having to remove the lid to keep heat and moisture locked in Lifetime limited warranty",1.000000,0.500000,...,78,100,100,100,100,-1.490450,1,1,0,0
69679,119478,romano 4ft. boxwood spiral topiari tree,topiari tree,1.00,topiary tree,Romano 4 ft. Boxwood Spiral Topiary Tree,enhanc home romano boxwood spiral topiari tree. wonder full bodi boxwood spiral topiari tree keep color rest lose color leaves. alway look forward rich green tone romano boxwood spiral topiari tree home. look high qualiti artifici tree make bold statement home light mobil design 700 leav indoor outdoor iron frame uv resist rubber leav 7in. pot enhanc room hous,"Enhance your home with a Romano Boxwood Spiral Topiary Tree. This wonderful full bodied Boxwood Spiral Topiary Tree keeps its colors when all the rest lose their color and leaves. You can always look forward to the rich green tones of 

In [61]:
tree_cols = [#'match_product_title', 'match_descr',
             'len_search_term_words', 'len_search_term_letters',
             'match_pos_title', 'match_brand', 'match_pos_descr', 'prod_title_ind', 'descr_ind',
             'words_std_title', 'words_std_descr', 'len_title', 'len_descr', 'last_word_title',
             'last_word_descr', 'match_orig_title', 'match_orig_descr',
             'fuzzy_title', 'fuzzy_descr', 'fuzzy_orig_title', 'fuzzy_orig_descr',
             'part_fuzzy_title', 'part_fuzzy_descr', 'part_fuzzy_orig_title', 'part_fuzzy_orig_descr',
             'first_word_title', 'first_word_descr',
             'match_numbers_title']

use_cols = tree_cols + ['search_term', 'product_title', 'descr', 'brand']

y = train['relevance'].values

clf = fitter()
X_train = clf.transform(train[use_cols])

gbm = lgb.LGBMRegressor(learning_rate=0.03, colsample_bytree=0.5, num_leaves=80, n_estimators=1000)
gbm.fit(X_train, y)

tc_len = len(tree_cols)
print(dict(zip(tree_cols, gbm.feature_importances_[:tc_len]) ))
print(gbm.score(X_train, y))


{'len_search_term_words': 245, 'len_search_term_letters': 662, 'match_pos_title': 198, 'match_brand': 63, 'match_pos_descr': 176, 'prod_title_ind': 580, 'descr_ind': 495, 'words_std_title': 241, 'words_std_descr': 287, 'len_title': 353, 'len_descr': 365, 'last_word_title': 102, 'last_word_descr': 58, 'match_orig_title': 111, 'match_orig_descr': 153, 'fuzzy_title': 660, 'fuzzy_descr': 480, 'fuzzy_orig_title': 488, 'fuzzy_orig_descr': 445, 'part_fuzzy_title': 188, 'part_fuzzy_descr': 97, 'part_fuzzy_orig_title': 220, 'part_fuzzy_orig_descr': 74, 'first_word_title': 87, 'first_word_descr': 41, 'match_numbers_title': 147}
0.6523918796360275


In [56]:
X_test = clf.transform(test[use_cols])
y_pred = gbm.predict(X_test)
y_pred[y_pred>3.] = 3.
y_pred[y_pred<1.] = 1.
results = pd.DataFrame({'id':test.index.values, 'relevance':y_pred})
results.to_csv('lgbm5.csv', header=True, index=False)

0.46884

In [57]:
## train
train['part_fuzzy_title'] = fuzzy_match_partial(train['search_term'], train['product_title'])
train['part_fuzzy_descr'] = fuzzy_match_partial(train['search_term'], train['descr'])

train['part_fuzzy_orig_title'] = fuzzy_match_partial(train['orig_search_term'], train['orig_product_title'])
train['part_fuzzy_orig_descr'] = fuzzy_match_partial(train['orig_search_term'], train['orig_descr'])
train.to_csv('zip/train5.csv.gz', compression='gzip')

## test
test['part_fuzzy_title'] = fuzzy_match_partial(test['search_term'], test['product_title'])
test['part_fuzzy_descr'] = fuzzy_match_partial(test['search_term'], test['descr'])

test['part_fuzzy_orig_title'] = fuzzy_match_partial(test['orig_search_term'], test['orig_product_title'])
test['part_fuzzy_orig_descr'] = fuzzy_match_partial(test['orig_search_term'], test['orig_descr'])
test.to_csv('zip/test5.csv.gz', compression='gzip')

In [62]:
tree_cols = ['match_product_title', 'match_descr', 'len_search_term_words', 'len_search_term_letters',
             'match_pos_title', 'match_brand', 'match_pos_descr', 'prod_title_ind', 'descr_ind',
             'words_std_title', 'words_std_descr', 'len_title', 'len_descr', 'last_word_title',
             'last_word_descr', 'match_orig_title', 'match_orig_descr',
             'fuzzy_title', 'fuzzy_descr', 'fuzzy_orig_title', 'fuzzy_orig_descr',
             'part_fuzzy_title', 'part_fuzzy_descr', 'part_fuzzy_orig_title', 'part_fuzzy_orig_descr',
             'first_word_title', 'first_word_descr',
             'match_numbers_title']

use_cols = tree_cols + ['search_term', 'product_title', 'descr', 'brand']

y = train['relevance'].values

clf = fitter()
X_train = clf.transform(train[use_cols])

gbm = lgb.LGBMRegressor(learning_rate=0.03, colsample_bytree=0.5, num_leaves=80, n_estimators=1000)
gbm.fit(X_train, y)

tc_len = len(tree_cols)
print(dict(zip(tree_cols, gbm.feature_importances_[:tc_len]) ))
print(gbm.score(X_train, y))

X_test = clf.transform(test[use_cols])
y_pred = gbm.predict(X_test)
y_pred[y_pred>3.] = 3.
y_pred[y_pred<1.] = 1.
results = pd.DataFrame({'id':test.index.values, 'relevance':y_pred})
results.to_csv('lgbm5.csv', header=True, index=False)

{'match_product_title': 249, 'match_descr': 153, 'len_search_term_words': 242, 'len_search_term_letters': 656, 'match_pos_title': 197, 'match_brand': 61, 'match_pos_descr': 148, 'prod_title_ind': 588, 'descr_ind': 469, 'words_std_title': 220, 'words_std_descr': 288, 'len_title': 378, 'len_descr': 341, 'last_word_title': 95, 'last_word_descr': 70, 'match_orig_title': 118, 'match_orig_descr': 150, 'fuzzy_title': 544, 'fuzzy_descr': 445, 'fuzzy_orig_title': 507, 'fuzzy_orig_descr': 357, 'part_fuzzy_title': 196, 'part_fuzzy_descr': 119, 'part_fuzzy_orig_title': 229, 'part_fuzzy_orig_descr': 84, 'first_word_title': 103, 'first_word_descr': 48, 'match_numbers_title': 138}
0.6514794888715436


0.46827

In [63]:
train['pred_diff'] = train['relevance'] - gbm.predict(X_train)
train.sort_values(['pred_diff'], ascending=False)[:50]

,product_uid,product_title,search_term,relevance,orig_search_term,orig_product_title,descr,orig_descr,match_product_title,match_descr,...,fuzzy_orig_descr,part_fuzzy_title,part_fuzzy_descr,part_fuzzy_orig_title,part_fuzzy_orig_descr,pred_diff,first_word_title,first_word_descr,match_numbers_title,match_numbers_descr
id,,,,,,,,,,,,,,,,,,,,,
203914,192088,behr premium plus 600e 2 harbor mist zero voc interior paint,locit 2 plus 1,3.00,locite 2 plus 1,BEHR Premium Plus #600E-2 Harbor Mist Zero VOC Interior Paint,behr premium plus zero voc self prime interior flat 100 acryl provid mildew resist finish. easi clean flat provid excel touch ultim durabl except hide. finish display non reflect matt appear california resid see proposit 65 inform ideal famili room live room dine room bedroom ceil 100 acryl finish easi clean tough durabl matt finish except hide 1 cover 400 sq. ft. depend color select surfac poros easi soap water clean except latex formula provid mildew resist finish self prime finish seal surfac actual paint color may vari screen printer represent onlin price includ paint care fee follow state ca co ct mn ri vt,"BEHR Premium Plus Zero VOC, Self-Priming Interior Flat is 100% acrylic and provides a mildew resistant finish. This easy-clean flat provides excellent touch-up, ultimate durability and exceptional hide. The finish displays a non-reflective, matte appearance California residents: see Proposition 65 information Ideal for family rooms, living rooms, dining rooms, bedrooms and ceilings 100% acrylic finish for easy cleaning Tough and durable matte finish Exceptional hide, 1 can covers up to 400 sq. ft. depending on color selected and surface porosity Easy soap and water clean-up Exceptional latex formula provides a mildew resistant finish Self priming finish that seals the surface Actual paint colors may vary from on-screen and printer representations Online Price includes Paint Care fee in the following states: CA, CO, CT, ME, MN, OR, RI, VT",0.500000,0.500000,...,57,100,100,100,100,1.139019,0,0,0,0
185512,178376,behr premium plus ultra 1 gal. p460 5 fiji semi gloss enamel exterior paint,fija,3.00,fija,BEHR Premium Plus Ultra 1-gal. #P460-5 Fiji Semi-Gloss Enamel Exterior Paint,perfect pop color give home serious curb appeal choos behr premium plus ultra semi gloss enamel exterior paint. radiant sheen creat sleek appear home exterior accent ideal trim shutter door includ garag door well window outdoor furniture. formul withstand wear also resist moistur fade stain california resid see proposit 65 inform suitabl applic wood vinyl stucco aluminum metal steel door window trim shutter garag door outdoor furnitur rail wrought iron 100 acryl base provid except adhes durabl extra protect shell guard damag caus xbyuv ray sever weather condit long last fresh paint look minim mainten enhanc mildew resist finish except hide 1 cover 400 sq. ft. depend color select surfac poros semi gloss enamel sheen offer radiant sleek appear formul easi clean soap water actual paint color may vari screen printer represent onlin price includ paint care fee follow state ca co ct mn ri vt,"For a perfect pop of color that will give your home some serious curb appeal, choose BEHR Premium Plus Ultra Semi-Gloss Enamel Exterior Paint. This radiant sheen will create a sleek appearance on your home's exterior accents and is ideal for trim, shutters and doors, including garage doors, as well as windows and outdoor furniture. It is formulated to withstand wear and will also resist moisture, fading and stains California residents: see Proposition 65 information Suitable for application on wood, vinyl, stucco, aluminum, metals, steel, doors, windows, trim, shutters, garage doors, outdoor furniture, railing and wrought iron 100% acrylic base provides exceptional adhesion and durability Extra-protective shell guards against damage caused by UV rays and severe weather conditions for a long-lasting freshly painted look with minimal maintenance Enhanced mildew res

In [64]:
train.sort_values(['pred_diff'], ascending=True)[:50]

,product_uid,product_title,search_term,relevance,orig_search_term,orig_product_title,descr,orig_descr,match_product_title,match_descr,...,fuzzy_orig_descr,part_fuzzy_title,part_fuzzy_descr,part_fuzzy_orig_title,part_fuzzy_orig_descr,pred_diff,first_word_title,first_word_descr,match_numbers_title,match_numbers_descr
id,,,,,,,,,,,,,,,,,,,,,
120624,140844,rachael ray 10 qt. cover stockpot,rachael ray,1.00,rachael ray,Rachael Ray 10 qt. Covered Stockpot,whether boil pasta make batch chili cook grain larg 10 quart stockpot right size. stovetop two short side handl help take less space cabinet. hard anod construct prefer even heat distribut throughout pan sides. nonstick cook surfac mean need worri food stick scrub dirti pan cleanup breeze. fact rachael stockpot dishwash safe ad conveni hard anod cookwar except durabl plus heat quick even reduc hot spot burn food clear coat exterior make cookwar dishwash safe conveni long last nonstick interior let food slide eas make cleanup quick easi fun rubber handl provid comfort grip oven safe 350deg. fahrenheit temper glass lid watch food cook without remov lid keep heat moistur lock lifetim limit warranti,"Whether you're boiling pasta, making a batch of chili or cooking grains, this large, 10 quart stockpot is just the right size. When it's not on the stovetop, its two short side handles help it take up less space in your cabinet. The hard-anodized construction is preferred for its even heat distribution throughout the pan and up the sides. The nonstick cooking surface means you don't need to worry about food sticking or about scrubbing dirty pans - cleanup is a breeze. In fact, Rachael's stockpot is dishwasher safe for added convenience Hard-anodized cookware is exceptionally durable, plus it heats quickly and evenly, reducing ""hot spots"" that can burn foods A clear coat exterior makes this cookware dishwasher safe for convenience Long-lasting nonstick interior lets food slide off with ease and makes cleanup quick and easy Fun rubberized handles provide a comfortable grip and are oven safe to 350 degrees Fahrenheit Tempered glass lid to watch your foods cook without having to remove the lid to keep heat and moisture locked in Lifetime limited warranty",1.000000,0.500000,...,78,100,100,100,100,-1.511105,1,1,0,0
136808,148897,earthquak 212cc tiller rear tine crt side shield,side shield,1.00,side shields,Earthquake 212cc Tiller Rear Tine CRT with Side Shields,earthquak 6015v rear tine rototil deliv ultim combin power size. rear mount tine till stubborn dirt easili high perform 212cc viper engine. engin compact rear tine rototil power larger counterpart easi singl hand control sod earthquak 6015v rear tine compact full size power compact frame durabl cast iron bronz gear drive transmiss,"The Earthquake 6015V rear tine rototiller delivers the ultimate combination of power and size. Rear-mounted tines till stubborn dirt easily with a high-performance 212cc Viper engine. We have engineered a compact rear tine rototiller with the same power as its larger counterparts Easy - single-handed control through sod Earthquake 6015V rear tine Compact - full-sized power in a compact frame Durable - cast iron, bronze gear drive transmission",1.000000,0.000000,...,7,100,55,100,58,-1.487367,1,0,0,0
151606,156981,ge 36in. rang microwav accessori filler kit slate,ge slate microwav,1.00,ge slate microwave,GE 36 in. Over-the-Range Microwave Accessory Filler Kit in Slate,36in. rang microwav trim kit provid custom appear built conveni ge ge profil microwave. ge applianc provid date technolog except qualiti simplifi way live. timeless appear famili applianc ideal family. come one trust name america know entir select applianc advanc practic microwav filler kit allow standard 30in. microwav fit 36in. open part includ 2 trim kit side mount 2 instal screw 1 connect bar kit best instal 12in. base cabinet 15in. cabinet instal kit j 15bumpfit select ge rang microwav jvm7195 dvm7195 jnm7196 pvm9195 pnm9196 pvm9215,"Your 36 

In [68]:
brand = pd.read_csv('zip/brand.csv.gz', index_col=['product_uid'], compression='gzip')
u_brands = brand['value'].str.lower().unique()

@np.vectorize
def brand_mentioned(search_term):
    if len(search_term) == 0:
        return 0
    for word in search_term.lower().split():
        if word in u_brands:
            return 1
    return 0

In [69]:
train['brand_mentioned'] = brand_mentioned(train['orig_search_term'])
test['brand_mentioned'] = brand_mentioned(test['orig_search_term'])

tree_cols = ['match_product_title', 'match_descr', 'len_search_term_words', 'len_search_term_letters',
             'match_pos_title', 'match_brand', 'match_pos_descr', 'prod_title_ind', 'descr_ind',
             'words_std_title', 'words_std_descr', 'len_title', 'len_descr', 'last_word_title',
             'last_word_descr', 'match_orig_title', 'match_orig_descr',
             'fuzzy_title', 'fuzzy_descr', 'fuzzy_orig_title', 'fuzzy_orig_descr',
             'part_fuzzy_title', 'part_fuzzy_descr', 'part_fuzzy_orig_title', 'part_fuzzy_orig_descr',
             'first_word_title', 'first_word_descr',
             'match_numbers_title', 'brand_mentioned']

use_cols = tree_cols + ['search_term', 'product_title', 'descr', 'brand']

y = train['relevance'].values

clf = fitter()
X_train = clf.transform(train[use_cols])

gbm = lgb.LGBMRegressor(learning_rate=0.03, colsample_bytree=0.5, num_leaves=80, n_estimators=1000)
gbm.fit(X_train, y)

tc_len = len(tree_cols)
print(dict(zip(tree_cols, gbm.feature_importances_[:tc_len]) ))
print(gbm.score(X_train, y))

X_test = clf.transform(test[use_cols])
y_pred = gbm.predict(X_test)
y_pred[y_pred>3.] = 3.
y_pred[y_pred<1.] = 1.
results = pd.DataFrame({'id':test.index.values, 'relevance':y_pred})
results.to_csv('lgbm5.csv', header=True, index=False)

{'match_product_title': 258, 'match_descr': 150, 'len_search_term_words': 228, 'len_search_term_letters': 646, 'match_pos_title': 168, 'match_brand': 64, 'match_pos_descr': 146, 'prod_title_ind': 562, 'descr_ind': 423, 'words_std_title': 244, 'words_std_descr': 305, 'len_title': 398, 'len_descr': 383, 'last_word_title': 96, 'last_word_descr': 51, 'match_orig_title': 112, 'match_orig_descr': 149, 'fuzzy_title': 570, 'fuzzy_descr': 439, 'fuzzy_orig_title': 525, 'fuzzy_orig_descr': 396, 'part_fuzzy_title': 163, 'part_fuzzy_descr': 102, 'part_fuzzy_orig_title': 224, 'part_fuzzy_orig_descr': 71, 'first_word_title': 93, 'first_word_descr': 42, 'match_numbers_title': 152, 'brand_mentioned': 15}
0.6524234348480967


In [71]:
attrs = pd.read_csv('zip/attributes.csv.gz', compression='gzip', index_col=['product_uid'], encoding=encoding)

/home/eugene/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [72]:
attrs.head()

,name,value
product_uid,,
100001.0,Bullet01,Versatile connector for various 90Â° connections and home repair projects
100001.0,Bullet02,Stronger than angled nailing or screw fastening alone
100001.0,Bullet03,Help ensure joints are consistently straight and strong
100001.0,Bullet04,Dimensions: 3 in. x 3 in. x 1-1/2 in.
100001.0,Bullet05,Made from 12-Gauge steel


In [75]:
other_attrs = attrs[attrs['name'] != 'MFG Brand Name']

In [83]:
attr_gr = other_attrs.groupby(['product_uid'])['value'].apply(lambda x: " ".join([str(a) for a in x]))
attr_gr.head()

product_uid
100001.0    Versatile connector for various 90Â° connections and home repair projects Stronger than angled nailing or screw fastening alone Help ensure joints are consistently straight and strong Dimensions: 3 in. x 3 in. x 1-1/2 in. Made from 12-Gauge steel Galvanized for extra corrosion resistance Install with 10d common nails or #9 x 1-1/2 in. Strong-Drive SD screws 12 Galvanized Steel 1 1.5 3 0.26 3                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
100002.0    Brush,Roller,Spray 6.63 in 7.76 in 6.63 in Revives wood and composite decks, railings, po

In [84]:
train['other_attrs'] = stem_sentence(train.product_uid.map(attr_gr))

In [85]:
train['match_other_attrs'] = fuzzy_match_partial(train['search_term'], train['other_attrs'])

In [86]:
test['other_attrs'] = stem_sentence(test.product_uid.map(attr_gr))
test['match_other_attrs'] = fuzzy_match_partial(test['search_term'], test['other_attrs'])

In [87]:
tree_cols = ['match_product_title', 'match_descr', 'len_search_term_words', 'len_search_term_letters',
             'match_pos_title', 'match_brand', 'match_pos_descr', 'prod_title_ind', 'descr_ind',
             'words_std_title', 'words_std_descr', 'len_title', 'len_descr', 'last_word_title',
             'last_word_descr', 'match_orig_title', 'match_orig_descr',
             'fuzzy_title', 'fuzzy_descr', 'fuzzy_orig_title', 'fuzzy_orig_descr',
             'part_fuzzy_title', 'part_fuzzy_descr', 'part_fuzzy_orig_title', 'part_fuzzy_orig_descr',
             'first_word_title', 'first_word_descr',
             'match_numbers_title', 'match_other_attrs']

use_cols = tree_cols + ['search_term', 'product_title', 'descr', 'brand']

y = train['relevance'].values

clf = fitter()
X_train = clf.transform(train[use_cols])

gbm = lgb.LGBMRegressor(learning_rate=0.03, colsample_bytree=0.5, num_leaves=80, n_estimators=1000)
gbm.fit(X_train, y)

tc_len = len(tree_cols)
print(dict(zip(tree_cols, gbm.feature_importances_[:tc_len]) ))
print(gbm.score(X_train, y))

X_test = clf.transform(test[use_cols])
y_pred = gbm.predict(X_test)
y_pred[y_pred>3.] = 3.
y_pred[y_pred<1.] = 1.
results = pd.DataFrame({'id':test.index.values, 'relevance':y_pred})
results.to_csv('lgbm5.csv', header=True, index=False)

{'match_product_title': 248, 'match_descr': 156, 'len_search_term_words': 235, 'len_search_term_letters': 621, 'match_pos_title': 154, 'match_brand': 67, 'match_pos_descr': 154, 'prod_title_ind': 554, 'descr_ind': 415, 'words_std_title': 245, 'words_std_descr': 292, 'len_title': 382, 'len_descr': 372, 'last_word_title': 95, 'last_word_descr': 61, 'match_orig_title': 120, 'match_orig_descr': 133, 'fuzzy_title': 551, 'fuzzy_descr': 387, 'fuzzy_orig_title': 534, 'fuzzy_orig_descr': 401, 'part_fuzzy_title': 162, 'part_fuzzy_descr': 124, 'part_fuzzy_orig_title': 218, 'part_fuzzy_orig_descr': 52, 'first_word_title': 88, 'first_word_descr': 33, 'match_numbers_title': 151, 'match_other_attrs': 133}
0.6530578855546139


0.46800

In [88]:
tree_cols = ['match_product_title', 'match_descr', 'len_search_term_words', 'len_search_term_letters',
             'match_pos_title', 'match_brand', 'match_pos_descr', 'prod_title_ind', 'descr_ind',
             'words_std_title', 'words_std_descr', 'len_title', 'len_descr',
             'last_word_title',
             'match_orig_title', 'match_orig_descr',
             'fuzzy_title', 'fuzzy_descr', 'fuzzy_orig_title', 'fuzzy_orig_descr',
             'part_fuzzy_title', 'part_fuzzy_descr', 'part_fuzzy_orig_title', 'part_fuzzy_orig_descr',
             'first_word_title',
             'match_numbers_title', 'match_other_attrs']

use_cols = tree_cols + ['search_term', 'product_title', 'descr', 'brand']

y = train['relevance'].values

clf = fitter()
X_train = clf.transform(train[use_cols])

gbm = lgb.LGBMRegressor(learning_rate=0.03, colsample_bytree=0.5, num_leaves=80, n_estimators=1000)
gbm.fit(X_train, y)

tc_len = len(tree_cols)
print(dict(zip(tree_cols, gbm.feature_importances_[:tc_len]) ))
print(gbm.score(X_train, y))

X_test = clf.transform(test[use_cols])
y_pred = gbm.predict(X_test)
y_pred[y_pred>3.] = 3.
y_pred[y_pred<1.] = 1.
results = pd.DataFrame({'id':test.index.values, 'relevance':y_pred})
results.to_csv('lgbm5.csv', header=True, index=False)

{'match_product_title': 261, 'match_descr': 164, 'len_search_term_words': 223, 'len_search_term_letters': 626, 'match_pos_title': 145, 'match_brand': 56, 'match_pos_descr': 186, 'prod_title_ind': 571, 'descr_ind': 469, 'words_std_title': 234, 'words_std_descr': 299, 'len_title': 366, 'len_descr': 384, 'last_word_title': 117, 'match_orig_title': 111, 'match_orig_descr': 154, 'fuzzy_title': 549, 'fuzzy_descr': 430, 'fuzzy_orig_title': 510, 'fuzzy_orig_descr': 425, 'part_fuzzy_title': 164, 'part_fuzzy_descr': 110, 'part_fuzzy_orig_title': 180, 'part_fuzzy_orig_descr': 73, 'first_word_title': 105, 'match_numbers_title': 142, 'match_other_attrs': 149}
0.6529445884049845


In [89]:
print(gbm.feature_importances_[tc_len:])

[423 378 315 278 342 293 279 275 273 310 253 240 313 317 277 247 264 323
 294 267 315 246 304 295 338 334 283 296 303 296 297 345 359 263 285 277
 282 335 311 277 313 305 319 304 305 268 325 328 320 321 307 301 261 303
 319 341 324 274 378 293 336 321 312 305 325 339 347 321 298 328 343 360
 352 355 342 308 401 325 347 364 416 275 259 304 297 287 261 259 270 327
 285 307 280 279 352 321 307 286 326 271 273 252 296 310 278 282 295 281
 299 252 291 293 290 273 313 280 310 328 268 259 299 316 300 290 312 275
 341 267 327 312 300 308 309 319 353 336 256 332 267 310 277 317 297 303
 301 337 336 298 324 300 345 281 325 301 353 337 350 299 329 348 487 249
 319 260 307 285 292 349 273 279 258 302 264 261 235 289 274 341 263 343
 300 310 335 279 310 292 293 307 331 297 304 297 272 308 302 290 374 316
 315 325 315 281 273 325 302 312 336 331 300 292 307 340 357 322 369 346
 296 348 313 347 231 160  99 144 212 118 118 124 132 143 118 135 130 137
 126 147 120 127 136 126 158 148 139 127 132 154 12

In [90]:
def fitter(tsvd1=80, tsvd2=80, tsvd3=60, tsvd=30):
    clf = Pipeline([
    ('union', FeatureUnion(
                transformer_list = [
                    ('cst',  cust_regression_vals()),  
                    ('txt1', Pipeline([('s1', cust_txt_col(col='search_term')),
                                       ('tfidf1', TfidfVectorizer(stop_words='english')),
                                       ('tsvd1', TruncatedSVD(n_components=80))])),
                    ('txt2', Pipeline([('s2', cust_txt_col(col='product_title')),
                                       ('tfidf2', TfidfVectorizer(stop_words='english')),
                                       ('tsvd2', TruncatedSVD(n_components=80))])),
                    ('txt3', Pipeline([('s3', cust_txt_col(col='descr')),
                                       ('tfidf3', TfidfVectorizer(stop_words='english')),
                                       ('tsvd3', TruncatedSVD(n_components=60))])),
                    ('txt4', Pipeline([('s4', cust_txt_col(col='brand')),
                                       ('tfidf4', TfidfVectorizer(stop_words='english')),
                                       ('tsvd4', TruncatedSVD(n_components=30))]))
                    ]
            ))])
    clf.fit(train[use_cols])
    return clf

In [91]:
tree_cols = ['match_product_title', 'match_descr', 'len_search_term_words', 'len_search_term_letters',
             'match_pos_title', 'match_brand', 'match_pos_descr', 'prod_title_ind', 'descr_ind',
             'words_std_title', 'words_std_descr', 'len_title', 'len_descr', 'last_word_title',
             'last_word_descr', 'match_orig_title', 'match_orig_descr',
             'fuzzy_title', 'fuzzy_descr', 'fuzzy_orig_title', 'fuzzy_orig_descr',
             'part_fuzzy_title', 'part_fuzzy_descr', 'part_fuzzy_orig_title', 'part_fuzzy_orig_descr',
             'first_word_title', 'first_word_descr',
             'match_numbers_title', 'match_other_attrs']

use_cols = tree_cols + ['search_term', 'product_title', 'descr', 'brand']

y = train['relevance'].values

clf = fitter(120, 120, 90)
X_train = clf.transform(train[use_cols])

gbm = lgb.LGBMRegressor(learning_rate=0.03, colsample_bytree=0.5, num_leaves=80, n_estimators=1000)
gbm.fit(X_train, y)

tc_len = len(tree_cols)
print(dict(zip(tree_cols, gbm.feature_importances_[:tc_len]) ))
print(gbm.feature_importances_[tc_len:])
print(gbm.score(X_train, y))

X_test = clf.transform(test[use_cols])
y_pred = gbm.predict(X_test)
y_pred[y_pred>3.] = 3.
y_pred[y_pred<1.] = 1.
results = pd.DataFrame({'id':test.index.values, 'relevance':y_pred})
results.to_csv('lgbm5.csv', header=True, index=False)

{'match_product_title': 253, 'match_descr': 176, 'len_search_term_words': 240, 'len_search_term_letters': 666, 'match_pos_title': 162, 'match_brand': 57, 'match_pos_descr': 134, 'prod_title_ind': 529, 'descr_ind': 450, 'words_std_title': 233, 'words_std_descr': 301, 'len_title': 396, 'len_descr': 339, 'last_word_title': 107, 'last_word_descr': 54, 'match_orig_title': 122, 'match_orig_descr': 133, 'fuzzy_title': 584, 'fuzzy_descr': 432, 'fuzzy_orig_title': 532, 'fuzzy_orig_descr': 396, 'part_fuzzy_title': 166, 'part_fuzzy_descr': 102, 'part_fuzzy_orig_title': 234, 'part_fuzzy_orig_descr': 64, 'first_word_title': 100, 'first_word_descr': 41, 'match_numbers_title': 150, 'match_other_attrs': 115}
[410 375 345 302 323 276 275 291 290 302 285 234 329 337 307 282 253 246
 283 273 309 258 299 325 308 300 308 285 292 298 342 289 338 332 327 285
 310 254 293 298 347 253 305 300 333 299 325 291 300 308 330 288 344 266
 304 284 351 265 352 278 330 408 327 381 306 273 351 329 339 345 320 377
 321 3

In [92]:
tree_cols = ['match_product_title', 'match_descr', 'len_search_term_words', 'len_search_term_letters',
             'match_pos_title', 'match_brand', 'match_pos_descr', 'prod_title_ind', 'descr_ind',
             'words_std_title', 'words_std_descr', 'len_title', 'len_descr', 'last_word_title',
             'last_word_descr', 'match_orig_title', 'match_orig_descr',
             'fuzzy_title', 'fuzzy_descr', 'fuzzy_orig_title', 'fuzzy_orig_descr',
             'part_fuzzy_title', 'part_fuzzy_descr', 'part_fuzzy_orig_title', 'part_fuzzy_orig_descr',
             'first_word_title', 'first_word_descr',
             'match_numbers_title', 'match_other_attrs']

use_cols = tree_cols + ['search_term', 'product_title', 'descr', 'brand']

y = train['relevance'].values

clf = fitter(60, 60, 45, 20)
X_train = clf.transform(train[use_cols])

gbm = lgb.LGBMRegressor(learning_rate=0.03, colsample_bytree=0.5, num_leaves=80, n_estimators=1000)
gbm.fit(X_train, y)

tc_len = len(tree_cols)
print(dict(zip(tree_cols, gbm.feature_importances_[:tc_len]) ))
print(gbm.feature_importances_[tc_len:])
print(gbm.score(X_train, y))

X_test = clf.transform(test[use_cols])
y_pred = gbm.predict(X_test)
y_pred[y_pred>3.] = 3.
y_pred[y_pred<1.] = 1.
results = pd.DataFrame({'id':test.index.values, 'relevance':y_pred})
results.to_csv('lgbm5.csv', header=True, index=False)

{'match_product_title': 287, 'match_descr': 178, 'len_search_term_words': 217, 'len_search_term_letters': 629, 'match_pos_title': 180, 'match_brand': 76, 'match_pos_descr': 170, 'prod_title_ind': 540, 'descr_ind': 513, 'words_std_title': 234, 'words_std_descr': 289, 'len_title': 360, 'len_descr': 357, 'last_word_title': 104, 'last_word_descr': 62, 'match_orig_title': 108, 'match_orig_descr': 137, 'fuzzy_title': 545, 'fuzzy_descr': 422, 'fuzzy_orig_title': 526, 'fuzzy_orig_descr': 353, 'part_fuzzy_title': 169, 'part_fuzzy_descr': 90, 'part_fuzzy_orig_title': 206, 'part_fuzzy_orig_descr': 77, 'first_word_title': 92, 'first_word_descr': 40, 'match_numbers_title': 156, 'match_other_attrs': 139}
[376 367 313 333 341 276 285 291 294 303 271 247 304 334 306 269 267 241
 274 289 359 280 279 286 365 294 285 323 324 268 315 255 328 293 296 249
 286 312 295 310 354 250 318 302 299 308 311 277 284 281 284 275 277 284
 283 347 285 298 260 296 354 350 346 307 326 252 296 290 346 341 359 364
 395 298

In [93]:
tree_cols = ['match_product_title', 'match_descr', 'len_search_term_words', 'len_search_term_letters',
             'match_pos_title', 'match_brand', 'match_pos_descr', 'prod_title_ind', 'descr_ind',
             'words_std_title', 'words_std_descr', 'len_title', 'len_descr', 'last_word_title',
             'last_word_descr', 'match_orig_title', 'match_orig_descr',
             'fuzzy_title', 'fuzzy_descr', 'fuzzy_orig_title', 'fuzzy_orig_descr',
             'part_fuzzy_title', 'part_fuzzy_descr', 'part_fuzzy_orig_title', 'part_fuzzy_orig_descr',
             'first_word_title', 'first_word_descr',
             'match_numbers_title', 'match_other_attrs']

use_cols = tree_cols + ['search_term', 'product_title', 'descr', 'brand']


clf = fitter(40, 40, 30, 15)
X_train = clf.transform(train[use_cols])

gbm = lgb.LGBMRegressor(learning_rate=0.03, colsample_bytree=0.5, num_leaves=80, n_estimators=1000)
gbm.fit(X_train, y)

tc_len = len(tree_cols)
print(dict(zip(tree_cols, gbm.feature_importances_[:tc_len]) ))
print(gbm.feature_importances_[tc_len:])
print(gbm.score(X_train, y))

X_test = clf.transform(test[use_cols])
y_pred = gbm.predict(X_test)
y_pred[y_pred>3.] = 3.
y_pred[y_pred<1.] = 1.
results = pd.DataFrame({'id':test.index.values, 'relevance':y_pred})
results.to_csv('lgbm5.csv', header=True, index=False)

{'match_product_title': 253, 'match_descr': 171, 'len_search_term_words': 234, 'len_search_term_letters': 586, 'match_pos_title': 170, 'match_brand': 70, 'match_pos_descr': 152, 'prod_title_ind': 515, 'descr_ind': 449, 'words_std_title': 249, 'words_std_descr': 298, 'len_title': 371, 'len_descr': 372, 'last_word_title': 119, 'last_word_descr': 58, 'match_orig_title': 113, 'match_orig_descr': 122, 'fuzzy_title': 568, 'fuzzy_descr': 451, 'fuzzy_orig_title': 514, 'fuzzy_orig_descr': 402, 'part_fuzzy_title': 170, 'part_fuzzy_descr': 106, 'part_fuzzy_orig_title': 207, 'part_fuzzy_orig_descr': 67, 'first_word_title': 88, 'first_word_descr': 42, 'match_numbers_title': 154, 'match_other_attrs': 122}
[485 360 313 323 369 300 282 324 312 309 239 245 293 342 309 280 251 298
 274 270 282 277 333 302 353 283 248 291 271 313 326 337 328 293 365 293
 340 288 315 291 346 268 314 307 257 288 320 309 305 351 302 278 318 354
 275 289 329 275 293 355 302 369 285 297 309 274 344 298 335 330 326 327
 363 33

0.46772

In [98]:
train['pred_diff'] = train['relevance'] - gbm.predict(X_train)
train.sort_values(['pred_diff'], ascending=False)[:50]

,product_uid,product_title,search_term,relevance,orig_search_term,orig_product_title,descr,orig_descr,match_product_title,match_descr,...,part_fuzzy_orig_title,part_fuzzy_orig_descr,pred_diff,first_word_title,first_word_descr,match_numbers_title,match_numbers_descr,brand_mentioned,other_attrs,match_other_attrs
id,,,,,,,,,,,,,,,,,,,,,
203914,192088,behr premium plus 600e 2 harbor mist zero voc interior paint,locit 2 plus 1,3.00,locite 2 plus 1,BEHR Premium Plus #600E-2 Harbor Mist Zero VOC Interior Paint,behr premium plus zero voc self prime interior flat 100 acryl provid mildew resist finish. easi clean flat provid excel touch ultim durabl except hide. finish display non reflect matt appear california resid see proposit 65 inform ideal famili room live room dine room bedroom ceil 100 acryl finish easi clean tough durabl matt finish except hide 1 cover 400 sq. ft. depend color select surfac poros easi soap water clean except latex formula provid mildew resist finish self prime finish seal surfac actual paint color may vari screen printer represent onlin price includ paint care fee follow state ca co ct mn ri vt,"BEHR Premium Plus Zero VOC, Self-Priming Interior Flat is 100% acrylic and provides a mildew resistant finish. This easy-clean flat provides excellent touch-up, ultimate durability and exceptional hide. The finish displays a non-reflective, matte appearance California residents: see Proposition 65 information Ideal for family rooms, living rooms, dining rooms, bedrooms and ceilings 100% acrylic finish for easy cleaning Tough and durable matte finish Exceptional hide, 1 can covers up to 400 sq. ft. depending on color selected and surface porosity Easy soap and water clean-up Exceptional latex formula provides a mildew resistant finish Self priming finish that seals the surface Actual paint colors may vary from on-screen and printer representations Online Price includes Paint Care fee in the following states: CA, CO, CT, ME, MN, OR, RI, VT",0.500000,0.500000,...,100,100,1.140383,0,0,0,0,0,ideal famili room live room dine room bedroom ceil 100 acryl finish easi clean tough durabl matt finish except hide 1 cover 400 sq. ft. depend color select surfac poros easi soap water clean except latex formula provid mildew resist finish self prime finish seal surfac actual paint color may vari screen printer represent onlin price includ paint care fee follow state ca co ct mn ri vt purpl lavend harbor mist 1 ga gallon 400 60 interior paint yes soap water mildew resist 208 215 229 flat matt 2 base 1 solid,100
193001,183692,lichtenberg white no. 918 millenni ryan heather textur sheer curtain panel 40in. w 95in. l,w g 918,3.00,w g 918,"LICHTENBERG White No. 918 Millennial Ryan Heathered Texture Sheer Curtain Panel, 40 in. W x 95 in. L",no. 918 millenni ryan heather textur semi sheer curtain casual solid add fresh finish touch decor setting. enhanc privaci allow light gentl filter through. clean simpl one pocket pole top design use standard decor curtain rod. mix match solid print look sheer panel gentl filter light header pole top panel machin washabl wide array color choos 100 polyest contain 1 curtain panel,"No. 918 Millennial Ryan heathered texture semi-sheer curtain is a casual solid that adds freshness and a finishing touch to any decor setting. Enhances privacy while allowing light to gently filter through. Clean, simple one-pocket pole top design can be used with a standard or decorative curtain rod. Mix and match with other solids and prints for a look that is all your own Sheer panel, gently filters light No header pole top panel Machine washable Wide array of colors to choose from 100% polyester Contains 1-curtain panel",0.666667,0.333333,...,100,100,1.051230,1,1,1,1,0,.1in. 95in. 40in. sheer panel gentl filter light header pole top panel machin washabl wide array color choos 100 polyest contain 1 curtain panel white white modern window treatment 100 polyest machin washabl rod pocket addit item includ indoor sheer yes polyest h

In [99]:
train.sort_values(['pred_diff'], ascending=True)[:50]

,product_uid,product_title,search_term,relevance,orig_search_term,orig_product_title,descr,orig_descr,match_product_title,match_descr,...,part_fuzzy_orig_title,part_fuzzy_orig_descr,pred_diff,first_word_title,first_word_descr,match_numbers_title,match_numbers_descr,brand_mentioned,other_attrs,match_other_attrs
id,,,,,,,,,,,,,,,,,,,,,
69679,119478,romano 4ft. boxwood spiral topiari tree,topiari tree,1.00,topiary tree,Romano 4 ft. Boxwood Spiral Topiary Tree,enhanc home romano boxwood spiral topiari tree. wonder full bodi boxwood spiral topiari tree keep color rest lose color leaves. alway look forward rich green tone romano boxwood spiral topiari tree home. look high qualiti artifici tree make bold statement home light mobil design 700 leav indoor outdoor iron frame uv resist rubber leav 7in. pot enhanc room hous,"Enhance your home with a Romano Boxwood Spiral Topiary Tree. This wonderful full bodied Boxwood Spiral Topiary Tree keeps its colors when all the rest lose their color and leaves. You can always look forward to the rich green tones of the Romano Boxwood Spiral Topiary Tree in your home. If you are looking for a High quality artificial tree to make a bold statement in your home, this is it Light and mobile design Over 700 leaves Indoor/outdoor Iron frame UV resistant rubber leaves 7 in. pot Enhances any room in your house",1.000000,1.000000,...,100,100,-1.539887,1,1,0,0,0,null,0
120624,140844,rachael ray 10 qt. cover stockpot,rachael ray,1.00,rachael ray,Rachael Ray 10 qt. Covered Stockpot,whether boil pasta make batch chili cook grain larg 10 quart stockpot right size. stovetop two short side handl help take less space cabinet. hard anod construct prefer even heat distribut throughout pan sides. nonstick cook surfac mean need worri food stick scrub dirti pan cleanup breeze. fact rachael stockpot dishwash safe ad conveni hard anod cookwar except durabl plus heat quick even reduc hot spot burn food clear coat exterior make cookwar dishwash safe conveni long last nonstick interior let food slide eas make cleanup quick easi fun rubber handl provid comfort grip oven safe 350deg. fahrenheit temper glass lid watch food cook without remov lid keep heat moistur lock lifetim limit warranti,"Whether you're boiling pasta, making a batch of chili or cooking grains, this large, 10 quart stockpot is just the right size. When it's not on the stovetop, its two short side handles help it take up less space in your cabinet. The hard-anodized construction is preferred for its even heat distribution throughout the pan and up the sides. The nonstick cooking surface means you don't need to worry about food sticking or about scrubbing dirty pans - cleanup is a breeze. In fact, Rachael's stockpot is dishwasher safe for added convenience Hard-anodized cookware is exceptionally durable, plus it heats quickly and evenly, reducing ""hot spots"" that can burn foods A clear coat exterior makes this cookware dishwasher safe for convenience Long-lasting nonstick interior lets food slide off with ease and makes cleanup quick and easy Fun rubberized handles provide a comfortable grip and are oven safe to 350 degrees Fahrenheit Tempered glass lid to watch your foods cook without having to remove the lid to keep heat and moisture locked in Lifetime limited warranty",1.000000,0.500000,...,100,100,-1.449115,1,1,0,0,0,null,25
136808,148897,earthquak 212cc tiller rear tine crt side shield,side shield,1.00,side shields,Earthquake 212cc Tiller Rear Tine CRT with Side Shields,earthquak 6015v rear tine rototil deliv ultim combin power size. rear mount tine till stubborn dirt easili high perform 212cc viper engine. engin compact rear tine rototil power larger counterpart easi singl hand control sod earthquak 6015v rear tine compact full size power compact frame durabl cast iron bronz gear drive transmiss,"The Earthquake 6015V rear tine rototiller delivers the ultimate combination of power and size. Rear-mounted tines till stubborn dirt easily with a high-performa

In [102]:
train[train['pred_diff'].abs().between(0.4, 0.8)]

,product_uid,product_title,search_term,relevance,orig_search_term,orig_product_title,descr,orig_descr,match_product_title,match_descr,...,part_fuzzy_orig_title,part_fuzzy_orig_descr,pred_diff,first_word_title,first_word_descr,match_numbers_title,match_numbers_descr,brand_mentioned,other_attrs,match_other_attrs
id,,,,,,,,,,,,,,,,,,,,,
37,100011,toro person pace recycl 22in. variabl speed self propel gas lawn mower brigg amp stratton engin,gas mow,3.00,gas mowe,Toro Personal Pace Recycler 22 in. Variable Speed Self-Propelled Gas Lawn Mower with Briggs &amp; Stratton Engine,recycl 22in. person pace variabl speed self propel gas lawn mower brigg stratton engine. homeown seek lawn mower high qualiti user friend design excel mulch capability. toro product known smart featur proven dependability. count recycl cut system front throw chamber atom blade lift suspend cut clip fine mulch healthi lush lawn 22in. steel deck atom blade brigg stratton 7.25ft. lbs. gross torqu 190cc engin readi start need prime choke person pace self propel system sens smooth adjust prefer walk pace 3 1 mulch side discharg bag bag demand quick switch mulch bag second xbyflip quick chang lever rear wheel drive self propel provid better traction cut condit let mower work washout port attach hose mower deck easili clean undersid deck quick connect bag system 9 cut posit 1 4in. cut height rang manicur lawn specif 3 year guarante start 2 year full warranti see retail detail gross torqu engin laboratori rate 2800 rpm per sae j1940 xbyth engin manufactur configur meet safeti emiss oper requir actual engin torqu class mower signific lower note product may vari xbystor home depot protect plan avail actual product offer may vari,"Recycler 22 in. Personal Pace Variable Speed Self-Propelled Gas Lawn Mower with Briggs & Stratton Engine. For homeowners seeking a lawn mower with high quality, a user-friendly design, and excellent mulching capability. Toro products are known for Smart Features and Proven Dependability. Count On It Recycler cutting system - front throw chamber and atomic blade lift, suspend and re-cut clippings into a fine mulch for a healthy, lush lawn 22 in. steel deck with atomic blade Briggs & Stratton 7.25 ft. lbs. gross torque 190cc engine with ready start - no need to prime or choke Personal pace self-propel system senses and smoothly adjusts to your preferred walking pace 3-in-1: mulch, side discharge or bag Bag-on-demand - quickly switch from mulching to bagging in seconds by flipping the quick-change lever Rear wheel drive self propel provides better traction in all cutting conditions - let the mower do the work Washout port - attach a hose to the mower deck to easily clean the underside of the deck""Quick connect"" bagging system 9 cutting positions with 1-4 in. cutting height range to manicure the lawn to your specifications 3-year guaranteed-to-start and 2-year full warranty, see retailer for details Gross torque of this engine was laboratory rated at 2800 rpm per SAE J1940 by the engine manufacturer, as configured to meet safety, emission, and operating requirements, the actual engine torque on this class of mower will be significantly lower Note: product may vary by store Home Depot Protection Plan Available Actual products offered may vary",0.500000,0.500000,...,100,100,0.553436,1,1,0,0,0,59in. 44in. 23in. 22in. recycl cut system front throw chamber atom blade lift suspend cut clip fine mulch healthi lush lawn 22in. steel deck atom blade brigg stratton 7.25ft. lbs. gross torqu 190cc engin readi start need prime choke person pace self propel system sens smooth adjust prefer walk pace 3 1 mulch side discharg bag bag demand quick switch mulch bag second xbyflip quick chang lever rear wheel drive self propel provid better traction cut condit let mower work washout port attach hose mower deck easili clean undersid deck quick connect bag system 9 cut posit 1 4in. cut height rang manicur lawn specif 3 year guarante start 2 year full warranti see retai

In [104]:
import spacy

nlp = spacy.load('en_core_web_sm')

In [107]:
@np.vectorize
def spacy_similarity(search_text, title):
    doc1 = nlp(search_text)
    doc2 = nlp(title)
    return doc1.similarity(doc2)

In [111]:
%%time
train['spacy_similarity'] = spacy_similarity(train['orig_search_term'], train['orig_product_title'])

CPU times: user 1h 36min 53s, sys: 2h 3min 6s, total: 3h 40min
Wall time: 18min 20s


In [112]:
%%time
test['spacy_similarity'] = spacy_similarity(test['orig_search_term'], test['orig_product_title'])

CPU times: user 3h 34min 58s, sys: 4h 33min 20s, total: 8h 8min 18s
Wall time: 40min 43s


In [ ]:
tree_cols = ['match_product_title', 'match_descr', 'len_search_term_words', 'len_search_term_letters',
             'match_pos_title', 'match_brand', 'match_pos_descr', 'prod_title_ind', 'descr_ind',
             'words_std_title', 'words_std_descr', 'len_title', 'len_descr', 'last_word_title',
             'last_word_descr', 'match_orig_title', 'match_orig_descr',
             'fuzzy_title', 'fuzzy_descr', 'fuzzy_orig_title', 'fuzzy_orig_descr',
             'part_fuzzy_title', 'part_fuzzy_descr', 'part_fuzzy_orig_title', 'part_fuzzy_orig_descr',
             'first_word_title', 'first_word_descr',
             'match_numbers_title', 'match_other_attrs', 'spacy_similarity']

use_cols = tree_cols + ['search_term', 'product_title', 'descr', 'brand']


clf = fitter(40, 40, 30, 15)
X_train = clf.transform(train[use_cols])

gbm = lgb.LGBMRegressor(learning_rate=0.02, colsample_bytree=0.5, num_leaves=80, n_estimators=1000)
gbm.fit(X_train, y)

tc_len = len(tree_cols)
print(dict(zip(tree_cols, gbm.feature_importances_[:tc_len]) ))
print(gbm.score(X_train, y))

X_test = clf.transform(test[use_cols])
y_pred = gbm.predict(X_test)
y_pred[y_pred>3.] = 3.
y_pred[y_pred<1.] = 1.
results = pd.DataFrame({'id':test.index.values, 'relevance':y_pred})
results.to_csv('lgbm5.csv', header=True, index=False)

In [114]:
train.to_csv('zip/train5.csv.gz', compression='gzip')
test.to_csv('zip/test5.csv.gz', compression='gzip')